In [24]:
import pandas as pd
import numpy as np
import copy

In [25]:
admissions = pd.read_csv(r'readmissions_data/ADMISSIONS.csv')

In [26]:
print(admissions.shape)
admissions.head()

(58976, 19)


,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [27]:
print(admissions.shape,admissions['SUBJECT_ID'].nunique())

(58976, 19) 46520


In [28]:
(admissions.groupby(['SUBJECT_ID'])['HADM_ID'].count()>1).sum()

7537

In [29]:
def get_readmitted_patients(admissions_in, readmission_days):
    admissions = copy.deepcopy(admissions_in)
    admissions['freq'] = admissions.groupby(['SUBJECT_ID'])['HADM_ID'].transform('count')
    admissions['ADMITTIME'] = pd.to_datetime(admissions['ADMITTIME'])
    admissions['DISCHTIME'] = pd.to_datetime(admissions['DISCHTIME'])
    admissions.sort_values(['SUBJECT_ID','ADMITTIME'],inplace=True)
    admissions['lag_diff'] = admissions.groupby(['SUBJECT_ID'])['ADMITTIME'].diff()
    admissions['readmn_days'] = admissions['lag_diff'].dt.days
    admissions.reset_index(drop = True, inplace = True)
    pat_ones = set(admissions[admissions['readmn_days']<=readmission_days].SUBJECT_ID)
    pat_zeros = set(admissions.SUBJECT_ID) - pat_ones
    return admissions, pat_ones, pat_zeros

In [30]:
admissions_mod, readmitted, not_readmitted = get_readmitted_patients(admissions, 30)

In [31]:
admissions_mod.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,freq,lag_diff,readmn_days
0,1,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,NaN,ASIAN,NaN,NaN,NEWBORN,0,1,1,NaT,NaN
1,2,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,MARRIED,WHITE,2101-10-20 17:09:00,2101-10-20 19:24:00,HYPOTENSION,0,1,1,NaT,NaN
2,3,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME WITH HOME IV PROVIDR,Private,...,SINGLE,WHITE,2191-03-15 13:10:00,2191-03-16 01:10:00,"FEVER,DEHYDRATION,FAILURE TO THRIVE",0,1,1,NaT,NaN
3,4,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,NaN,ASIAN,NaN,NaN,NEWBORN,0,1,1,NaT,NaN
4,5,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,...,MARRIED,WHITE,NaN,NaN,CHRONIC RENAL FAILURE/SDA,0,1,1,NaT,NaN


In [55]:
admissions_mod.isna().sum()

ROW_ID                      0
SUBJECT_ID                  0
HADM_ID                     0
ADMITTIME                   0
DISCHTIME                   0
DEATHTIME               53122
ADMISSION_TYPE              0
ADMISSION_LOCATION          0
DISCHARGE_LOCATION          0
INSURANCE                   0
LANGUAGE                25332
RELIGION                  458
MARITAL_STATUS          10128
ETHNICITY                   0
EDREGTIME               28099
EDOUTTIME               28099
DIAGNOSIS                  25
HOSPITAL_EXPIRE_FLAG        0
HAS_CHARTEVENTS_DATA        0
freq                        0
lag_diff                46520
readmn_days             46520
dtype: int64

In [44]:
def grpby_drop_pat(df, pat_list, readmission_days):
    if df['SUBJECT_ID'].tolist()[0] not in pat_list:
        admission_id = df['HADM_ID'].tolist()
        return pd.DataFrame.from_dict({'switch_flag':[0], 'HADM_ID_lst':[admission_id]})
    else:
        readmission_days_list = df['readmn_days'].fillna(9999).tolist()
        for i in range(len(readmission_days_list)-1):
            if readmission_days_list[i]> readmission_days:
                x = 0
            else:
                x = 1

            if readmission_days_list[i+1]> readmission_days:
                y = 0
            else:
                y = 1
            delta = y - x
            if delta <0:
                return pd.DataFrame.from_dict({'switch_flag':[2]})
        admission_id =   df[df['HADM_ID'] != df['HADM_ID'].max()]['HADM_ID'].tolist()
    return pd.DataFrame.from_dict({'switch_flag':[1], 'HADM_ID_lst':[admission_id]})

In [45]:
df_target = admissions_mod.groupby('SUBJECT_ID').apply(grpby_drop_pat, pat_list = readmitted, readmission_days = 30)
df_target.reset_index(inplace = True)
df_target.drop(columns = ['level_1'],inplace = True)

In [46]:
a = set(df_target[df_target.switch_flag == 1].SUBJECT_ID).union(set(df_target[df_target.switch_flag == 2].SUBJECT_ID))

In [47]:
len(set(df_target[df_target.switch_flag == 2].SUBJECT_ID))

540

In [48]:
len(a), len(readmitted)

(2084, 2084)

In [49]:
df_target.rename(columns={'SUBJECT_ID':'pat_id'}, inplace = True)

In [53]:
df_target.to_pickle('target_30.pkl')

In [54]:
admissions_mod[admissions_mod.SUBJECT_ID==362]

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,freq,lag_diff,readmn_days
465,466,362,142749,2112-07-10 02:31:00,2112-07-16 13:45:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,WIDOWED,WHITE,2112-07-09 21:22:00,2112-07-10 04:18:00,HYPONATREMIA-R/O MYOCARDIAL INFARCTION-RHABDOM...,0,1,2,NaT,NaN
466,467,362,150873,2112-07-28 17:08:00,2112-08-03 17:56:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,WIDOWED,WHITE,2112-07-28 15:43:00,2112-07-28 18:29:00,SUBDURAL HEMATOMA,0,1,2,18 days 14:37:00,18.0
